# Part 4: Preparing the Target Labels

## Goal
This is the final step of data preparation. In Part 3, I created the "Features" (the inputs). Now, I need to format the "Labels" (the crop yields we want to predict).

Here is the plan:
1.  **Load** the features and the raw yield data.
2.  **Pivot** the yield data so that each crop gets its own column (e.g., `Y_rice`, `Y_wheat`).
3.  **Merge** the Features (X) and Labels (Y) into one final dataset.
4.  **Save** everything so it's ready for the Machine Learning model.

In [52]:
import pandas as pd
import numpy as np

### 1. Loading the Data
I'll start by loading the features file I made in the previous notebook (`x_features`) and the original raw yield data.

In [53]:
# Loading the Features from Part 3
X = pd.read_parquet('Parquet/x_features_v3.parquet')

# Loading the raw yield data
label_yield = pd.read_parquet('Parquet/label_yield.parquet')

### 2. Cleaning Crop Names
Just like before, I need to make sure the crop names are clean and consistent (lowercase, no spaces) so they match perfectly when I create columns for them.

In [54]:
# Remove special characters and spaces
label_yield['item'] = label_yield['item'].str.replace(r'[^0-9a-zA-Z ]', '', regex=True)
label_yield['item'] = label_yield['item'].str.replace(" ", "_").str.lower()

label_yield.head()

,area,item,year,label
0,Afghanistan,maize_corn,1970-12-31,1475.7
1,Afghanistan,maize_corn,1971-12-31,1340.0
2,Afghanistan,maize_corn,1972-12-31,1565.2
3,Afghanistan,maize_corn,1973-12-31,1617.0
4,Afghanistan,maize_corn,1974-12-31,1617.0


### 3. Pivoting the Data
Currently, the data is in "long" format (one row per crop). I need to pivot it to "wide" format so that every row represents one Year/Country, and the crops are separate columns.

In [55]:
# Make sure year is an integer
label_yield['year'] = pd.to_datetime(label_yield['year']).dt.year

# Pivot the table
Y = label_yield.pivot_table(
    index=['year','area'],  # The rows
    columns='item',         # The new columns
    values='label'          # The values to fill in
).reset_index()

### 4. Renaming Columns
To make things clear, I'll add a `Y_` prefix to the crop columns. This way, when I merge everything, I'll easily know that `Y_rice` is the target variable I'm trying to predict.

In [56]:
# Get current columns
current_cols = Y.columns.tolist()

# Filter out the index columns
crop_cols = [c for c in current_cols if c not in ['year', 'area']]

# Rename: Keep year/area as is, add Y_ to the crops
new_col_names = ['year', 'area'] + [f'Y_{c}' for c in crop_cols]
Y.columns = new_col_names

Y.head()

,year,area,Y_bananas,Y_barley,Y_cassava_fresh,Y_cucumbers_and_gherkins,Y_maize_corn,Y_oil_palm_fruit,Y_other_vegetables_fresh_nec,Y_potatoes,Y_rice,Y_soya_beans,Y_sugar_beet,Y_sugar_cane,Y_tomatoes,Y_watermelons,Y_wheat
0,1970,Afghanistan,NaN,1174.6,NaN,NaN,1475.7,NaN,6127.8,9536.4,1811.9,NaN,14090.9,22000.0,NaN,7229.4,956.3
1,1970,Albania,NaN,1077.8,NaN,NaN,2071.8,NaN,12278.3,5469.3,2970.5,NaN,23638.9,NaN,12333.3,NaN,1537.7
2,1970,Algeria,NaN,668.5,NaN,NaN,1023.5,NaN,4891.3,6254.2,1581.0,NaN,19719.9,NaN,9449.6,8977.0,624.5
3,1970,Angola,10000.0,NaN,3555.6,NaN,912.0,9523.8,6515.7,6296.3,1198.0,NaN,NaN,50932.6,3076.9,NaN,854.6
4,1970,Antigua and Barbuda,1500.0,NaN,4000.0,4615.4,2400.0,NaN,6250.0,NaN,NaN,NaN,NaN,37272.7,3437.5,NaN,NaN


### 5. Merging Features and Targets
Now I just combine the Features (X) and the Labels (Y) using an inner join. This ensures that every row in the final dataset has both the weather/input data AND the yield data.

In [57]:
XY = X.merge(Y, on=['year', 'area'], how='inner')

# Checking the final size
print(f"Final dataset shape: {XY.shape}")

Final dataset shape: (6589, 75)


### 6. Saving the Final Files
I'll save the final dataset as a Parquet file (for speed) and also as a CSV file as requested.

In [58]:
# Save to Parquet
XY.to_parquet('Parquet/XY_v3.parquet')

# Save to CSV
XY.to_csv('Data/final_ML_dataset.csv', index=False)

print("Files saved successfully.")

Files saved successfully.


In [59]:
# Show a sample (Thailand) to double check the columns
pd.set_option('display.max_columns', None)

XY[XY['area'] == 'China, mainland'].head(20)

,year,area,avg_yield_maize_corn_1y,avg_yield_maize_corn_2y,avg_yield_maize_corn_3y,avg_yield_other_vegetables_fresh_nec_1y,avg_yield_other_vegetables_fresh_nec_2y,avg_yield_other_vegetables_fresh_nec_3y,avg_yield_potatoes_1y,avg_yield_potatoes_2y,avg_yield_potatoes_3y,avg_yield_rice_1y,avg_yield_rice_2y,avg_yield_rice_3y,avg_yield_sugar_cane_1y,avg_yield_sugar_cane_2y,avg_yield_sugar_cane_3y,avg_yield_wheat_1y,avg_yield_wheat_2y,avg_yield_wheat_3y,avg_yield_oil_palm_fruit_1y,avg_yield_oil_palm_fruit_2y,avg_yield_oil_palm_fruit_3y,avg_yield_barley_1y,avg_yield_barley_2y,avg_yield_barley_3y,avg_yield_soya_beans_1y,avg_yield_soya_beans_2y,avg_yield_soya_beans_3y,avg_yield_sugar_beet_1y,avg_yield_sugar_beet_2y,avg_yield_sugar_beet_3y,avg_yield_watermelons_1y,avg_yield_watermelons_2y,avg_yield_watermelons_3y,avg_yield_cucumbers_and_gherkins_1y,avg_yield_cucumbers_and_gherkins_2y,avg_yield_cucumbers_and_gherkins_3y,avg_yield_tomatoes_1y,avg_yield_tomatoes_2y,avg_yield_tomatoes_3y,avg_yield_bananas_1y,avg_yield_bananas_2y,avg_yield_bananas_3y,avg_yield_cassava_fresh_1y,avg_yield_cassava_fresh_2y,avg_yield_cassava_fresh_3y,rain_annual,rain_sin,rain_cos,solar_annual,solar_sin,solar_cos,temp_annual,temp_sin,temp_cos,pesticides_lag1,fertilizer_lag1,latitude,longitude,Y_bananas,Y_barley,Y_cassava_fresh,Y_cucumbers_and_gherkins,Y_maize_corn,Y_oil_palm_fruit,Y_other_vegetables_fresh_nec,Y_potatoes,Y_rice,Y_soya_beans,Y_sugar_beet,Y_sugar_cane,Y_tomatoes,Y_watermelons,Y_wheat
1346,1982,"China, mainland",3051.3,3065.10,3038.466667,18008.0,17087.15,16852.433333,10279.2,10761.35,10886.400000,4328.0,4230.90,4236.633333,53863.5,50738.65,47833.833333,2109.1,2000.25,2046.666667,NaN,NaN,NaN,2583.3,2382.10,2417.933333,1163.4,1131.65,1097.933333,14600.6,14427.80,12804.166667,16397.3,15855.35,15839.833333,12000.0,11602.95,11600.033333,22826.1,22608.70,22479.866667,14000.0,13130.00,13817.966667,15217.4,14909.60,14606.400000,763.66,-154.249206,-390.791077,NaN,NaN,NaN,1.715000,-25.425118,-56.315518,NaN,156.550263,35.0,103.0,15461.5,2807.0,15319.1,11707.3,3269.1,NaN,16974.8,10889.8,4891.3,1073.7,14543.9,56506.8,22500.0,14777.8,2451.7
1347,1983,"China, mainland",3269.1,3160.20,3133.100000,16974.8,17491.40,17049.700000,10889.8,10584.50,10804.166667,4891.3,4609.65,4451.033333,56506.8,55185.15,52661.366667,2451.7,2280.40,2150.733333,NaN,NaN,NaN,2807.0,2695.15,2523.733333,1073.7,1118.55,1112.333333,14543.9,14572.25,14466.500000,14777.8,15587.55,15496.166667,11707.3,11853.65,11637.733333,22500.0,22663.05,22572.466667,15461.5,14730.75,13907.166667,15319.1,15268.25,15046.100000,373.92,-150.207421,-147.599971,NaN,NaN,NaN,2.114167,-30.921604,-54.170596,NaN,161.214356,35.0,103.0,15514.3,2500.0,15833.3,11904.8,3623.5,NaN,19765.7,11375.4,5096.1,1291.1,16901.2,47655.4,23265.3,16000.0,2801.7
1348,1984,"China, mainland",3623.5,3446.30,3314.633333,19765.7,18370.25,18249.500000,11375.4,11132.60,10848.133333,5096.1,4993.70,4771.800000,47655.4,52081.10,52675.233333,2801.7,2626.70,2454.166667,NaN,NaN,NaN,2500.0,2653.50,2630.100000,1291.1,1182.40,1176.066667,16901.2,15722.55,15348.566667,16000.0,15388.90,15725.033333,11904.8,11806.05,11870.700000,23265.3,22882.65,22863.800000,15514.3,15487.90,14991.933333,15833.3,15576.20,15456.600000,498.32,-121.770022,-209.564926,NaN,NaN,NaN,1.499167,-37.270309,-55.281210,NaN,173.656104,35.0,103.0,15000.0,2750.0,15833.3,12500.0,3960.2,NaN,21904.8,11654.2,5372.6,1330.6,16518.4,54284.4,23200.0,15932.9,2969.1
1349,1985,"China, mainland",3960.2,3791.85,3617.600000,21904.8,20835.25,19548.433333,11654.2,11514.80,11306.466667,5372.6,5234.35,5120.000000,54284.4,50969.90,52815.533333,2969.1,2885.40,2740.833333,NaN,NaN,NaN,2750.0,2625.00,2685.666667,1330.6,1310.85,1231.800000,16518.4,16709.80,15987.833333,15932.9,15966.45,15570.233333,12500.0,12202.40,12037.366667,23200.0,23232.65,22988.433333,15000.0,15257.15,15325.266667,15833.3,15833.30,15661.900000,503.52,-129.600966,-276.504565,15.560000,0.482853,-33.275618,1.980833,-29.518631,-56.336140,NaN,175.9